In [51]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
from scipy.ndimage.interpolation import zoom

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import pandas as pd

In [53]:
base = '/home/michael/Dropbox/engage/FairbrassFirmanetal_/data/predictions/ACI_baseline/'
df = pd.read_csv(base + 'overlap_noRound.csv')

In [54]:
savedir = '/media/michael/Engage/data/audio/alison_data/golden_set/predictions/aci_baseline/biotic/per_file_predictions/'

if not os.path.exists(savedir):
    os.makedirs(savedir)

for key in df.keys()[1:]:
    
    # loading ACI pred and gt
    preds = df[key]
    gt, _ = pickle.load(open(savepath.replace('aci_baseline', 'mel32_large_test_golden')))
    
    # converting to correct shape and format
    factor = float(gt.shape[0]) / preds.shape[0]
    preds_conv = zoom(preds, factor, order=1)
    preds_conv += preds_conv.min()
    preds_conv /= preds_conv.max()
    preds_conv = np.vstack((preds_conv, 1.0 - preds_conv)).T
    
    # saving to disk
    savepath = savedir + key.replace('.wav', '.pkl')
    with open(savepath, 'w') as f:
        pickle.dump((gt, preds_conv), f, -1)